In [1]:
import pandas as pd
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.compat import lzip

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
sales_train_evaluation = pd.read_csv('sales_train_evaluation.csv')

In [4]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')

In [5]:
sell_prices=pd.read_csv('sell_prices.csv')

In [6]:
sell_prices.head

<bound method NDFrame.head of         store_id        item_id  wm_yr_wk  sell_price
0           CA_1  HOBBIES_1_001     11325        9.58
1           CA_1  HOBBIES_1_001     11326        9.58
2           CA_1  HOBBIES_1_001     11327        8.26
3           CA_1  HOBBIES_1_001     11328        8.26
4           CA_1  HOBBIES_1_001     11329        8.26
...          ...            ...       ...         ...
6841116     WI_3    FOODS_3_827     11617        1.00
6841117     WI_3    FOODS_3_827     11618        1.00
6841118     WI_3    FOODS_3_827     11619        1.00
6841119     WI_3    FOODS_3_827     11620        1.00
6841120     WI_3    FOODS_3_827     11621        1.00

[6841121 rows x 4 columns]>

In [7]:
sales_train_validation.head

<bound method NDFrame.head of                                   id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_validation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_validation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_validation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_validation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_validation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1

In [8]:


sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [195]:
df=pd.read_csv('sales_train_validation.csv')

In [196]:
data=df

In [197]:
def cal_days(data): #calculate and output a list of days 
    start_day=1
    day_format="d_"
    converted=False
    cols=[]
    for j in range(math.ceil((len(data)-6)//7) ):
        for i in range(1,8):
            cols.append(day_format+str(start_day))
            start_day=start_day+1
    return cols


    

In [198]:
len(cols)

1911

In [199]:
def create_week(cols,df): # create columns for each week and quantity of products sold that week
    for i in range(0,len(cols),7):
        temp=cols[i:i+7]

        df["week" +str(int(i/7+1))]=df[temp].sum(axis=1)
    

In [200]:
create_week(cols,df)
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,week264,week265,week266,week267,week268,week269,week270,week271,week272,week273
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,6,8,10,6,6,10,3,10,5,9
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,3,1,2,3,0,0,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,1,0,10,3,1,1,2,5,7
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,18,7,10,14,16,15,10,15,14
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,12,7,6,8,9,10,9,7,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,4,1,3,5,0,0,0,0,4,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,4,4,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,4,2,11,12,9,4,2,9,10,4
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,12,0,8,4,10,7,9,7,7,5


In [201]:
def cal_extra_days(data):# calcualte extra days not included in weeks listed
    if not ((len(data.columns)-6)/7).is_integer():
        extra_days=(len(data.columns)-6)-(((len(data.columns)-6)//7)*7)
    return extra_days

In [202]:
extra_days=cal_extra_days(data)
extra_days

2

In [203]:
def add_extra_days_col(data,cols): # add extra days to list to sum all later to week
    temp_col=[]
    extra_days=cal_extra_days(data) 
    for i in range(((len(cols))//7)*7+1,((len(cols))//7)*7+extra_days+1):
        temp="d_"+str(i)
        temp_col.append(temp)   
    
    data["week" +str((len(cols))//7+1)]=data[temp_col].sum(axis=1)

In [205]:
add_extra_days_col(df,cols)
df

KeyError: "['d_1914'] not in index"

In [206]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,week265,week266,week267,week268,week269,week270,week271,week272,week273,week274
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,10,6,6,10,3,10,5,9,2
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,3,1,2,3,0,0,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,10,3,1,1,2,5,7,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18,7,10,14,16,15,10,15,14,9
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,7,6,8,9,10,9,7,8,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,3,5,0,0,0,0,4,1,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,4,4,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,11,12,9,4,2,9,10,4,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,8,4,10,7,9,7,7,5,4


In [207]:
def create_week_sales_info(df,cols):
    info_sales=df.iloc[:,:6]
    week_sales=df.iloc[:,-math.ceil((len(cols))/7):]
    result_df = pd.concat([info_sales, week_sales], axis=1)
    return result_df

In [208]:
result_df=create_week_sales_info(df,cols)
result_df

In [291]:
result_df

,id,item_id,dept_id,cat_id,store_id,state_id,week2,week3,week4,week5,...,week265,week266,week267,week268,week269,week270,week271,week272,week273,week274
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,10,6,6,10,3,10,5,9,2
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,3,1,2,3,0,0,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,10,3,1,1,2,5,7,2
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18,7,10,14,16,15,10,15,14,9
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,7,6,8,9,10,9,7,8,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,16,10,41,38,...,1,3,5,0,0,0,0,4,1,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11,7,6,6,...,0,0,0,0,0,0,4,4,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,35,55,24,15,...,2,11,12,9,4,2,9,10,4,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,8,4,10,7,9,7,7,5,4


In [224]:
def create_data_predict(result,sell_data):
    start_week="week"+str((int(str(sell_data.iloc[0,2])[2])*52-52+int(str(sell_data.iloc[0,2])[3:])))
    x=result_df.loc[1][start_week:].values.tolist()
    y=sell_data.iloc[0:146,3].values.tolist()

    return x,y

In [225]:
x,y=create_data_predict(result_df,sell_prices)

In [226]:
len(x)

146

In [227]:
y

[9.58,
 9.58,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,
 8.26,

In [60]:
x=result_df.loc[1][start_week:]

In [327]:
y=sell_prices.iloc[0:146,3]

In [328]:
y

0      9.58
1      9.58
2      8.26
3      8.26
4      8.26
       ... 
141    8.26
142    8.26
143    8.38
144    8.38
145    8.38
Name: sell_price, Length: 146, dtype: float64

In [76]:
data = pd.concat([x,y], axis=1,ignore_index=False)

In [229]:
x= sm.add_constant(x)

In [96]:
model = sm.OLS(y,x)

In [97]:
results = model.fit()

In [98]:
results.params

array([ 8.28248642e+00, -9.62646042e-04])

In [ ]:
results.tvalues

In [ ]:
print(results.t_test([1, 0]))

In [109]:
model=sm.OLS(y, x).fit()

In [230]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                   0.02564
Date:                Wed, 06 Mar 2024   Prob (F-statistic):              0.873
Time:                        10:37:11   Log-Likelihood:                 65.905
No. Observations:                 146   AIC:                            -127.8
Df Residuals:                     144   BIC:                            -121.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.2825      0.018    469.362      0.000       8.248       8.317
x1            -0.0010      0.006     -0.160      0.873      -0.013       0.011
==============================================================================
Omnibus:                      256.038   Durbin-Watson:                   0.509
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28398.698
Skew:                           8.221   Prob(JB):                         0.00
Kurtosis:                      69.317   Cond. No.                         4.27
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
plt.rc("figure", figsize=(16, 8))


In [ ]:
plt.rc("font", size=14)

In [ ]:
import seaborn as sns

In [ ]:
p=model.params

In [ ]:
p[0]

In [79]:
data

,quantity,sell_price
0,1,9.58
1,3,9.58
2,0,8.26
3,2,8.26
4,2,8.26
...,...,...
141,3,8.26
142,0,8.26
143,0,8.38
144,1,8.38


In [78]:
data.rename(columns={1:"quantity"},inplace=True)

In [29]:
from linearmodels import IV2SLS 

In [125]:
iv_reg = IV2SLS.from_formula("sell_price ~ quantity", data).fit()


In [126]:
iv_reg.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                            OLS Estimation Summary                            
==============================================================================
Dep. Variable:             sell_price   R-squared:                      0.0162
Estimator:                        OLS   Adj. R-squared:                -0.0567
No. Observations:                 146   F-statistic:                   -0.2551
Date:                Tue, Mar 05 2024   P-value (F-stat)                1.0000
Time:                        19:36:43   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
==================================================================================
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
Intercept          8.2680     0.0055     1512.9     0.0000      8.2573      8.2787
quantity[T.1]      0.0240     0.0296     0.8115     0.4171     -0.0340      0.0820
quantity[T.2]     -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.3]      0.0448     0.0520     0.8612     0.3891     -0.0572      0.1468
quantity[T.4]     -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.5]     -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.6]     -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.7]     -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.9]     -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.13]    -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
quantity[T.14]    -0.0080     0.0055    -1.4639     0.1432     -0.0187      0.0027
==================================================================================
"""

In [245]:
test=sell_prices.loc[0:146]
test[]

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
142,CA_1,HOBBIES_1_001,11610,8.26
143,CA_1,HOBBIES_1_001,11611,8.38
144,CA_1,HOBBIES_1_001,11612,8.38
145,CA_1,HOBBIES_1_001,11613,8.38


In [255]:
price_range=sell_prices.groupby(['store_id','item_id'])['sell_price'].apply(lambda x: max(x) - min(x))
max_change_product=price_range.idxmax()
max_change_value=price_range.max()

In [270]:
max_change_product

('WI_3', 'HOUSEHOLD_2_406')

In [272]:
price_range.max()

104.05999999999999

In [269]:
max_change_value

104.05999999999999

In [344]:
test=sell_prices.loc[sell_prices["store_id"]=="WI_3" ]

In [345]:
test=test.loc[sell_prices["item_id"]=="HOUSEHOLD_2_406"]

In [282]:
test

,store_id,item_id,wm_yr_wk,sell_price
6485826,WI_3,HOUSEHOLD_2_406,11102,12.47
6485827,WI_3,HOUSEHOLD_2_406,11103,12.47
6485828,WI_3,HOUSEHOLD_2_406,11104,12.47
6485829,WI_3,HOUSEHOLD_2_406,11105,12.47
6485830,WI_3,HOUSEHOLD_2_406,11106,12.47
...,...,...,...,...
6486102,WI_3,HOUSEHOLD_2_406,11617,12.46
6486103,WI_3,HOUSEHOLD_2_406,11618,12.46
6486104,WI_3,HOUSEHOLD_2_406,11619,12.46
6486105,WI_3,HOUSEHOLD_2_406,11620,12.46


In [284]:
display(test.to_string())

'        store_id          item_id  wm_yr_wk  sell_price\n6485826     WI_3  HOUSEHOLD_2_406     11102       12.47\n6485827     WI_3  HOUSEHOLD_2_406     11103       12.47\n6485828     WI_3  HOUSEHOLD_2_406     11104       12.47\n6485829     WI_3  HOUSEHOLD_2_406     11105       12.47\n6485830     WI_3  HOUSEHOLD_2_406     11106       12.47\n6485831     WI_3  HOUSEHOLD_2_406     11107       12.47\n6485832     WI_3  HOUSEHOLD_2_406     11108       12.47\n6485833     WI_3  HOUSEHOLD_2_406     11109       12.47\n6485834     WI_3  HOUSEHOLD_2_406     11110       12.47\n6485835     WI_3  HOUSEHOLD_2_406     11111       12.47\n6485836     WI_3  HOUSEHOLD_2_406     11112       12.47\n6485837     WI_3  HOUSEHOLD_2_406     11113       12.47\n6485838     WI_3  HOUSEHOLD_2_406     11114       12.47\n6485839     WI_3  HOUSEHOLD_2_406     11115       12.47\n6485840     WI_3  HOUSEHOLD_2_406     11116       12.47\n6485841     WI_3  HOUSEHOLD_2_406     11117       12.47\n6485842     WI_3  HOUSEHOLD_2_

In [287]:
dates=test.loc[test["sell_price"] >13]["wm_yr_wk"]

In [295]:
start_week="week"+str((int(str(sell_prices.iloc[0,2])[2])*52-52+int(str(sell_prices.iloc[0,2])[3:])))
x=result_df.loc[1][start_week:].values.tolist()
y=sell_prices.iloc[0:146,3].values.tolist()

In [306]:
start_week=dates.iloc[0]


In [309]:
start_week
x=result_df.loc[1][start_week:].values.tolist()


In [318]:
result=result_df.loc[result_df["item_id"]=="HOUSEHOLD_2_406"]

In [320]:
result=result.loc[result["store_id"]=="WI_3"]

In [355]:
x=result.iloc[:,6:].values.tolist()

In [356]:
x=x[0]

In [364]:
x[115:125]

[0, 5, 2, 2, 1, 0, 0, 1, 2, 0]

In [ ]:
y=sell_prices.iloc[0:146,3].values.tolist()

In [346]:
test=test["sell_price"].values.tolist()

In [358]:
test[121]

107.32

In [349]:
x= sm.add_constant(x)

In [353]:
sm.OLS(test[0:-8],x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.4219
Date:                Wed, 06 Mar 2024   Prob (F-statistic):              0.517
Time:                        11:46:06   Log-Likelihood:                -1018.3
No. Observations:                 273   AIC:                             2041.
Df Residuals:                     271   BIC:                             2048.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.5041      0.835     16.164      0.000      11.859      15.149
x1            -0.2854      0.439     -0.650      0.517      -1.150       0.580
==============================================================================
Omnibus:                      448.838   Durbin-Watson:                   0.654
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            75832.290
Skew:                           8.834   Prob(JB):                         0.00
Kurtosis:                      82.714   Cond. No.                         2.97
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""